In [16]:
import gymnasium as gym
import highway_env

# Agent
from stable_baselines3 import DQN


import sys
from tqdm.notebook import trange

from stable_baselines3 import DQN
import pprint
from matplotlib import pyplot as plt
import numpy as np
import joblib
from tqdm import trange

In [17]:
import base64
from pathlib import Path

from gymnasium.wrappers import RecordVideo
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display



def record_videos(env, video_folder="videos"):
    wrapped = RecordVideo(
        env, video_folder=video_folder, episode_trigger=lambda e: True
    )

    # Capture intermediate frames
    env.unwrapped.set_record_video_wrapper(wrapped)

    return wrapped


def show_videos(path="videos"):
    html = []
    for mp4 in Path(path).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


modify obs

In [18]:
def extract_features_from_dataset(row,prev_action):
    """
    Extract features from the dataset based on the given criteria.
    """
    processed_data = []

    # Ego vehicle features
    ego_features = row[:5]
    ego_lane = ego_features[2] // 4 + 1  # Lane ID of ego vehicle
    ego_speed = ego_features[3]  # Speed of ego vehicle

    # Other vehicles' features
    other_vehicles = row[5:50].reshape(9, 5)  # 9 vehicles, 5 features each
    
    # Separate features of other vehicles
    other_lanes = other_vehicles[:, 2] // 4 + 1  # Lane IDs of other vehicles
    distances = np.abs(other_vehicles[:, 1] - ego_features[1])  # Distances from ego vehicle
    relative_velocities = other_vehicles[:, 3] - ego_speed  # Relative velocities

    # Number of vehicles in ego lane and adjacent lanes
    vehicles_in_ego_lane = np.sum(other_lanes == ego_lane)
    vehicles_in_left_lane = np.sum(other_lanes == ego_lane - 1)
    vehicles_in_right_lane = np.sum(other_lanes == ego_lane + 1)

    ## Closest vehicles
    closest_ego_index = np.where(other_lanes == ego_lane, distances, np.inf).argmin() if vehicles_in_ego_lane != 0 else np.nan
    closest_left_index = np.where(other_lanes == ego_lane - 1, distances, np.inf).argmin() if vehicles_in_left_lane != 0 else np.nan
    closest_right_index = np.where(other_lanes == ego_lane + 1, distances, np.inf).argmin() if vehicles_in_right_lane != 0 else np.nan

    # Distances of other vehicles
    ## Ego lane
    if np.isnan(closest_ego_index):
        closest_in_ego_lane_dist = 10000  # Assign large value for no vehicle
        relative_velocity_ego_lane = 10000 
    else:
        closest_in_ego_lane_dist = distances[closest_ego_index]
        relative_velocity_ego_lane = relative_velocities[closest_ego_index]
    
    ## Left lane
    if np.isnan(closest_left_index):
        # Check if the left lane is non-existent (i.e., topmost lane)
        if ego_lane == 1:
            closest_left_lane_dist = 0  # No lane to the left of topmost lane
            relative_velocity_left_lane = 0  # No vehicle in non-existent lane
        else:
            closest_left_lane_dist = 10000  # No vehicle in left lane
            relative_velocity_left_lane = 10000  # No vehicle in left lane
    else:
        closest_left_lane_dist = distances[closest_left_index]
        relative_velocity_left_lane = relative_velocities[closest_left_index]
    
    ## Right lane
    if np.isnan(closest_right_index):
        # Check if the right lane is non-existent (i.e., bottommost lane)
        if ego_lane == 4:
            closest_right_lane_dist = 0  # No lane to the right of bottommost lane
            relative_velocity_right_lane = 0  # No vehicle in non-existent lane
        else:
            closest_right_lane_dist = 10000  # No vehicle in right lane
            relative_velocity_right_lane = 10000  # No vehicle in right lane
    else:
        closest_right_lane_dist = distances[closest_right_index]
        relative_velocity_right_lane = relative_velocities[closest_right_index]

    prev_action = prev_action

    # Append computed features
    processed_data.append([
        vehicles_in_ego_lane,
        vehicles_in_left_lane,
        vehicles_in_right_lane,
        closest_in_ego_lane_dist,
        closest_left_lane_dist,
        closest_right_lane_dist,
        relative_velocity_ego_lane,
        relative_velocity_left_lane,
        relative_velocity_right_lane,
        prev_action
    ])

    return np.array(processed_data)

In [19]:
# def extract_features_from_dataset(row):
#     """
#     Extract features from the dataset based on the given criteria.
#     """
#     processed_data = []

#     # Ego vehicle features
#     ego_features = row[:5]
#     ego_lane = ego_features[2]//4 + 1  # Lane ID of ego vehicle
#     ego_speed = ego_features[3]  # Speed of ego vehicle

#     # Other vehicles' features
#     other_vehicles = row[5:50].reshape(9, 5)  # 9 vehicles, 5 features each
#     # actions = row[50]
#     # Separate features of other vehicles
#     other_lanes = other_vehicles[:, 2] // 4 + 1  # Lane IDs of other vehicles
#     distances = np.abs(other_vehicles[:, 1] - ego_features[1])  # Distances from ego vehicle
#     relative_velocities = other_vehicles[:, 3] - ego_speed  # Relative velocities

#     # Number of vehicles in ego lane and adjacent lanes
#     vehicles_in_ego_lane = np.sum(other_lanes == ego_lane)
#     vehicles_in_left_lane = np.sum(other_lanes == ego_lane - 1)
#     vehicles_in_right_lane = np.sum(other_lanes == ego_lane + 1)

#     ## Closest vehicles
#     closest_ego_index = np.where(other_lanes == ego_lane, distances, np.inf).argmin() if vehicles_in_ego_lane !=0 else np.NAN
#     closest_left_index = np.where(other_lanes == ego_lane - 1, distances, np.inf).argmin() if vehicles_in_left_lane !=0 else np.NAN
#     closest_right_index = np.where(other_lanes == ego_lane + 1, distances, np.inf).argmin() if vehicles_in_right_lane !=0 else np.NAN

#     # Distances of other vehicles
#     ##ego
#     if np.isnan(closest_ego_index):
#         closest_in_ego_lane_dist = 10000  # Assign large value for no vehicle
#     else:
#         closest_in_ego_lane_dist = distances[closest_ego_index]  
#     ##left
#     if np.isnan(closest_left_index):
#         closest_left_lane_dist = 10000  # Assign default value for no vehicle
#     else:
#         closest_left_lane_dist = distances[closest_left_index]  
#     ##right
#     if np.isnan(closest_right_index):
#         closest_right_lane_dist = 10000  # Assign default value for no vehicle
#     else:
#         closest_right_lane_dist = distances[closest_right_index]  

#     ##relative velocities of closest vehicles
#     ##ego
#     if np.isnan(closest_ego_index):
#         relative_velocity_ego_lane = 10000  # Assign large value for no vehicle
#     else:
#         relative_velocity_ego_lane = relative_velocities[closest_ego_index]  
#     ##left
#     if np.isnan(closest_left_index):
#         relative_velocity_left_lane = 10000  # Assign large value for no vehicle
#     else:
#         relative_velocity_left_lane = relative_velocities[closest_left_index]  
#     ##right
#     if np.isnan(closest_right_index):
#         relative_velocity_right_lane = 10000  # Assign large value for no vehicle
#     else:
#         relative_velocity_right_lane = relative_velocities[closest_right_index]  
    
# # Append computed features
#     processed_data.append([
#         vehicles_in_ego_lane,
#         vehicles_in_left_lane,
#         vehicles_in_right_lane,
#         closest_in_ego_lane_dist,
#         closest_left_lane_dist,
#         closest_right_lane_dist,
#         relative_velocity_ego_lane,
#         relative_velocity_left_lane,
#         relative_velocity_right_lane
#     ])

#     return np.array(processed_data)

generate evaluation dataset

In [20]:
def process(obs,prev_action):
    ##load model
    #rf_model = joblib.load("models_try/rf_test_no_smote")
    #print(f"Model loaded")
    ##
    obs_flat = obs.flatten() # Flatten and reshape observation
  
    obs_processed = extract_features_from_dataset(obs_flat,prev_action)
    #processed_obs.append(obs_processed)
    print(obs_processed)


In [34]:
def rf_query(obs, prev_action):
    # Load the models
    rf_model_binary = joblib.load('models_try/binary_rf_model_collision_free_upsampled.pkl')
    rf_model_major = joblib.load("models_try/major_rf_model_down_upsampled.pkl")
    rf_model_minor = joblib.load("models_try/minor_rf_model_upsampled.pkl")
    
    obs_flat = obs.flatten()  # Flatten and reshape observation
    obs_processed = extract_features_from_dataset(obs_flat, prev_action)
    
    # Get the binary prediction and probabilities
    binary_pred = rf_model_binary.predict(obs_processed)[0]
    binary_pred_prob = rf_model_binary.predict_proba(obs_processed)[0]
    
    # Get the minor or major class prediction and probabilities
    if binary_pred == 0:
        minor_pred = rf_model_minor.predict(obs_processed)[0]
        minor_pred_prob = rf_model_minor.predict_proba(obs_processed)[0]
        return [binary_pred, binary_pred_prob, minor_pred, minor_pred_prob]
    else:
        major_pred = rf_model_major.predict(obs_processed)[0]
        major_pred_prob = rf_model_major.predict_proba(obs_processed)[0]
        return [binary_pred, binary_pred_prob, major_pred, major_pred_prob]

In [46]:
import os
import gymnasium as gym
import numpy as np
from gymnasium.wrappers import RecordVideo
from tqdm import trange  # For progress tracking
from IPython.display import display, Video

# Base setting
vehicleCount = 10

# Environment configuration
config = {
   "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "vehicles_count": vehicleCount,
                "see_behind": True,
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 1.55,
            'ego_Spacing': 0,
            "show_trajectories": True,
            "render_agent": True,}

# Create directory for video storage if it doesn't exist
video_dir = 'videos/rf_tests2'
if not os.path.exists(video_dir):
    os.makedirs(video_dir)

# Create directory for action predictions storage if it doesn't exist
predictions_dir = 'predictions/rf_tests2'
if not os.path.exists(predictions_dir):
    os.makedirs(predictions_dir)

# Create and configure environment
env = gym.make('highway-v0', render_mode='rgb_array', config=config)

# Wrap environment with video recording
env = RecordVideo(env, video_folder=video_dir, episode_trigger=lambda episode_id: True)

prev_action = 4
steps = 0
# Run test episodes and record them
for episode in trange(5, desc='Test episodes'):
    # Prepare to log the actions taken in this episode
    actions_file_path = os.path.join(predictions_dir, f'episode_{episode}_actions_normal.txt')
    
    with open(actions_file_path, 'w') as action_file:
        (obs, info), done, truncated = env.reset(), False, False
        while not (done or truncated):
            Class, Class_prob, action, action_prob = rf_query(obs, prev_action)
            obs, reward, done, truncated, info = env.step(int(action))
            steps += 1
            
            # Log the action and probabilities
            action_file.write(f"Class = {Class}  (Probability: {Class_prob})  Action: {action} (Probability: {action_prob})\n")
        
            prev_action = action
        print(steps)
    
env.close()


# Display the recorded videos
show_videos()


/home/prachit/.local/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/rf_tests2 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
Test episodes:   0%|          | 0/5 [00:00<?, ?it/s]

/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/pytho

Moviepy - Building video /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/rf_tests2/rl-video-episode-0.mp4.
Moviepy - Writing video /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/rf_tests2/rl-video-episode-0.mp4



Test episodes:  20%|██        | 1/5 [00:11<00:46, 11.69s/it]

Moviepy - Done !
Moviepy - video ready /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/rf_tests2/rl-video-episode-0.mp4
17


...

Moviepy - Done !
Moviepy - video ready /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/rf_tests2/rl-video-episode-3.mp4
137


/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/prachit/.local/lib/pytho

Moviepy - Building video /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/rf_tests2/rl-video-episode-4.mp4.
Moviepy - Writing video /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/rf_tests2/rl-video-episode-4.mp4



Test episodes: 100%|██████████| 5/5 [02:00<00:00, 24.16s/it]


Moviepy - Done !
Moviepy - video ready /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/rf_tests2/rl-video-episode-4.mp4
177
